# variables

In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup

headers = json.loads(r'''{
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "trctestcookie=ok; trctestcookie=ok; bblastvisit=1517316262; bblastactivity=0; __utmz=60066354.1517316266.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __gads=ID=0150c2db39515748:T=1517316267:S=ALNI_MaDEROky928259c08-Wiv9vdVzIMA; trc_cookie_storage=taboola%2520global%253Auser-id%3D9fb09c65-d038-47cc-aca3-9186c0df7852-tuct1686a87; __utma=60066354.2080145337.1517316266.1519810697.1519820804.11; __utmc=60066354; __utmt=1; __utmb=60066354.123.0.1519822641336",
    "Host": "www.dogforum.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36"
}''')

## Function

In [2]:
def getID():
    infos = soup.select('strong')
    return re.findall('<a href="http://www.dogforum.com/members/(.+)/.+ onclick', str(infos))[0]

In [3]:
def getName():
    infos = soup.select('strong')
    return re.findall("\t(.+)'s Profile", infos[0].text)[0]

In [4]:
def setStatistics():
    for info in infos:
        if(re.findall('Total Posts: (.+)', info.text)):
            totalPosts = int(re.findall('Total Posts: (.+)', info.text)[0].replace(',', ''))
        elif(re.findall('Posts Per Day: (.+)', info.text)):
            postPerDay = float("{0:.2f}".format(float(re.findall('Posts Per Day: (.+)', info.text)[0])))
        elif(re.findall('Join Date: (.+)', info.text)):
            joinDate = re.findall('Join Date: (.+)', info.text)[0]
        elif(re.findall('Referrals: (.+)', info.text)):
            referrals = int(re.findall('Referrals: (.+)', info.text)[0])
        elif(re.findall('Last Activity: (.+)', info.text)):
            lastActivity = re.findall('Last Activity: (.+)', info.text)[0]
            
    try:
        dataDic['TotalPosts'] = totalPosts
        dataDic['PostsPerDay'] = postPerDay
        dataDic['JoinDate'] = joinDate
        dataDic['Referrals'] = referrals
        try:
            dataDic['LastActivity'] = lastActivity
        except:
            print('NO lastActivity field~')
    except:
        print('setStatistics() ERROR!')

In [5]:
def setAbout():
    try:
        about = '<dt class="shade">' + re.findall('<dt class="shade">(.+)</dl>', str(infos))[0]
    except:
        return
    aboutSoup = BeautifulSoup(about, 'lxml')
    item = []
    value = []
    # add data
    for i in aboutSoup.select('dt.shade'):
        item.append(i.text)
    for j in aboutSoup.select('dd'):
        value.append(j.text)
    # add Signature
    item.append('Signature')
    try:
        value.append(soup.select('dl.list_no_decoration > dd#signature')[0].text.strip())
    except:
        return
    
    # iterate it & add it into aboutDic
    for i in range(0, len(item)):
        dataDic[item[i]] = value[i]

In [6]:
def setMemberDic(name, ID):
    memberDic['content'] = dataDic
    memberDic['id'] = ID
    memberDic['name'] = name

## Use Function
- use for loop in urls

In [44]:
# url = 'http://www.dogforum.com/members/128001/'
# url = 'http://www.dogforum.com/members/165089/'
url = 'http://www.dogforum.com/members/1/'

In [45]:
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')
infos = soup.select('ul.list_no_decoration li')
dataDic = {}
# all data store in here
memberDic = {}

In [7]:
def getMemberDic(url):
    ID = getID()
    name = getName()
    
    setStatistics()
    setAbout()
    setMemberDic(name, ID)
    
    return memberDic

In [47]:
getMemberDic(url)

NO lastActivity field~


{'content': {'JoinDate': '10-07-2008',
  'PostsPerDay': 0.59,
  'Referrals': 1,
  'TotalPosts': 2027},
 'id': '1',
 'name': 'Administrator'}

In [24]:
# svalue = ['JoinDate', 'LastActivity', 'PostsPerDay', 'Referrals', 'TotalPosts']

### Function:
- connect to rethinkdb
- Get links

In [8]:
import rethinkdb as r
conn = r.connect(db='Dogforum')

In [9]:
tables = r.table_list().run(conn)
tables

['Breeds',
 'DogDetail',
 'Dog_Health',
 'Food',
 'Gear_supplies',
 'General_Discussion',
 'Grooming',
 'Introductions',
 'MemberInfos',
 'Memorials',
 'New_Additions',
 'Perform_sports',
 'Train_Behavior']

In [10]:
combine_url = []

In [11]:
for table_num in range(0, len(tables)):
    cursor = r.table(tables[table_num]).pluck('author_url').distinct().run(conn)
    for num in range(0, len(cursor)):
        for url in cursor[num].values():
            combine_url.append(url)

In [12]:
len(combine_url)

37520

In [13]:
member_url = set(combine_url)
len(member_url)

20504

In [16]:
list(member_url)[:6]

['http://www.dogforum.com/members/10094/',
 'http://www.dogforum.com/members/114706/',
 'http://www.dogforum.com/members/187745/',
 'http://www.dogforum.com/members/104514/',
 'http://www.dogforum.com/members/191417/',
 'http://www.dogforum.com/members/11630/']

## Function
- store data into Member

In [15]:
for url in member_url:
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'lxml')
    infos = soup.select('ul.list_no_decoration li')
    dataDic = {}
    # all data store in here
    memberDic = {}
    try:
        post = getMemberDic(url)
        res = r.db('Dogforum').table('MemberInfos').insert(post).run(conn)
    except:
        print(url + '  ERROR!!')

NO lastActivity field~
NO lastActivity field~
NO lastActivity field~
http://www.dogforum.com/members/8580/  ERROR!!
http://www.dogforum.com/members/161753/  ERROR!!
http://www.dogforum.com/members/5366/  ERROR!!
http://www.dogforum.com/members/11917/  ERROR!!
http://www.dogforum.com/members/3563/  ERROR!!
http://www.dogforum.com/members/61642/  ERROR!!
http://www.dogforum.com/members/129386/  ERROR!!


- Crawler failed link again

In [9]:
failedLink = ['http://www.dogforum.com/members/8580/', 'http://www.dogforum.com/members/161753/', 'http://www.dogforum.com/members/5366/', 'http://www.dogforum.com/members/11917/', 'http://www.dogforum.com/members/3563/', 'http://www.dogforum.com/members/61642/', 'http://www.dogforum.com/members/129386/']

In [10]:
for url in failedLink:
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'lxml')
    infos = soup.select('ul.list_no_decoration li')
    dataDic = {}
    memberDic = {}
    try:
        post = getMemberDic(url)
        res = r.db('Dogforum').table('MemberInfos').insert(post).run(conn)
    except:
        print(url + '  ERROR!!')

### Json透過以下方法取值

In [25]:
# 字典列表
for k in memberDic[name][0].items():
    print(k[0] + ": " + k[1])

ID: 165089
TotalPosts: 1
PostsPerDay: 0.00
LastActivity: 08-15-2016 10:05 PM
JoinDate: 08-15-2016
Referrals: 0


In [26]:
for k in memberDic[name][1].items():
    print(k[0] + ": " + k[1])

Biography: Proud owner of a Pembroke Welsh Corgi. :)
Location: Los Angeles
Signature: Hello
